In [1]:
!pip install transformers
!pip install datasets
!pip install sentence_transformers
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
import pandas as pd
from datasets import load_dataset
import pandas as pd
import nltk
nltk.download('punkt')

from sentence_transformers import SentenceTransformer, util
from summarizer import Summarizer,TransformerSummarizer
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading dataset

In [3]:
data=pd.read_csv("Hotel_Reviews.csv")
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397.0,1403.0,Only the park outside of the hotel was beauti...,11.0,7.0,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0.0,1403.0,No real complaints the hotel was great great ...,105.0,7.0,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42.0,1403.0,Location was good and staff were ok It is cut...,21.0,9.0,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210.0,1403.0,Great location in nice surroundings the bar a...,26.0,1.0,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140.0,1403.0,Amazing location and building Romantic setting,8.0,3.0,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259964 entries, 0 to 259963
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               259964 non-null  object 
 1   Additional_Number_of_Scoring                259964 non-null  int64  
 2   Review_Date                                 259964 non-null  object 
 3   Average_Score                               259964 non-null  float64
 4   Hotel_Name                                  259964 non-null  object 
 5   Reviewer_Nationality                        259964 non-null  object 
 6   Negative_Review                             259963 non-null  object 
 7   Review_Total_Negative_Word_Counts           259963 non-null  float64
 8   Total_Number_of_Reviews                     259963 non-null  float64
 9   Positive_Review                             259963 non-null  object 
 

Creating Reviews dataframe

In [5]:
#pos/negative review counter
data["pos_count"] = 1
data["neg_count"] = 1
data["pos_count"] = data.apply(lambda x: 0 if x["Positive_Review"] == 'No Positive' else x["pos_count"],axis =1)
data["neg_count"] = data.apply(lambda x: 0 if x["Negative_Review"] == 'No Negative' else x["neg_count"],axis =1)

In [6]:
pos_reviews=data[data.pos_count>0].iloc[:,9]
neg_reviews=data[data.neg_count>0].iloc[:,6]
pos_reviews

0          Only the park outside of the hotel was beauti...
1          No real complaints the hotel was great great ...
2          Location was good and staff were ok It is cut...
3          Great location in nice surroundings the bar a...
4           Amazing location and building Romantic setting 
                                ...                        
259959     Staff was very helpful and accommodating reco...
259960     Quiet interior rooms staff made afternoon tea...
259961     The staff were extremely helpful in organizin...
259962     Fantastic Location Staff were extremely welco...
259963                                                  NaN
Name: Positive_Review, Length: 241387, dtype: object

In [7]:
pos_reviews=pos_reviews.to_frame().iloc[:40000,]
pos_reviews["Labels"]=1
pos_reviews.columns=["Review","Labels"]
neg_reviews=neg_reviews.to_frame().iloc[:40000,]
neg_reviews["Labels"]=0
neg_reviews.columns=["Review","Labels"]

In [8]:
reviews=pd.concat([neg_reviews, pos_reviews]).reset_index(drop=True).dropna()
reviews

,Review,Labels
0,I am so angry that i made this post available...,0
1,Rooms are nice but for elderly a bit difficul...,0
2,My room was dirty and I was afraid to walk ba...,0
3,You When I booked with your company on line y...,0
4,Backyard of the hotel is total mess shouldn t...,0
...,...,...
79995,Friendly staff Great breakfast Plenty of towe...,1
79996,great breakfast,1
79997,Location cleanliness breakfasts,1
79998,Comfortable room Good selection at breakfast ...,1


Creating words corpus

In [9]:
words = ""
frequencies = dict()
for review in reviews['Review']:
  words += str(review) + " "

words = [word.lower() for word in nltk.word_tokenize(words) if word.isalpha()]

for word in words:
  if word not in frequencies:
    frequencies[word] = 1
  else:
    frequencies[word] += 1
words_sorted = sorted(frequencies, key=frequencies.get, reverse=True)
words_sorted

['the',
 'and',
 'was',
 'to',
 'a',
 'room',
 'in',
 'very',
 'of',
 'staff',
 'for',
 'i',
 'location',
 'hotel',
 'we',
 'it',
 'were',
 'is',
 'not',
 'breakfast',
 'good',
 'with',
 'but',
 'on',
 'great',
 'at',
 't',
 'had',
 'friendly',
 'small',
 'bed',
 'that',
 'no',
 'have',
 'from',
 'as',
 'rooms',
 'helpful',
 'our',
 'so',
 'there',
 'my',
 'this',
 'you',
 'clean',
 'nice',
 'comfortable',
 'be',
 'all',
 'they',
 'excellent',
 'are',
 'would',
 'nothing',
 'stay',
 'bathroom',
 'which',
 'service',
 'could',
 'when',
 'really',
 'out',
 'too',
 'bar',
 'one',
 'us',
 'lovely',
 'only',
 'an',
 'shower',
 's',
 'london',
 'night',
 'didn',
 'like',
 'bit',
 'little',
 'also',
 'everything',
 'just',
 'reception',
 'close',
 'well',
 'if',
 'been',
 'station',
 'or',
 'more',
 'get',
 'up',
 'time',
 'me',
 'by',
 'view',
 'did',
 'food',
 'area',
 'can',
 'check',
 'walk',
 'perfect',
 'price',
 'even',
 'floor',
 'comfy',
 'quiet',
 'again',
 'facilities',
 'some',
 '

In [10]:
def search(word):
  auto = []
  count = 0
  for key in words_sorted:
    if (word in key):
      count += 1
      auto.append(key)
      if (count == 2):
        return auto

In [11]:
def get_ngrams(n, tokens):
    grams = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
    return grams

In [12]:
import random
import copy

class NgramModel(object):

    def __init__(self, n):
        self.n = n

        self.context = {}

        self.ngram_counter = {}

    def update(self, tokens):
        """
        Updates Language Model
        """
        n = self.n
        ngrams = get_ngrams(n, tokens)
        for ngram in ngrams:
            if ngram in self.ngram_counter:
                self.ngram_counter[ngram] += 1.0
            else:
                self.ngram_counter[ngram] = 1.0

            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]
    def prob(self, context, token):
        """
        Calculates probability of a candidate token to be generated given a context
        """
        try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context

        except KeyError:
            result = 0.0
        return result

    def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        """
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
            summ += map_to_probs[token]
            if summ > r:
                return token

    def generate_text(self, token_count, starting):
        """
        :param token_count: number of words to be produced
        """
        n = self.n
        context_queue = copy.deepcopy(starting)
        result = []
        for _ in range(token_count):
            obj = self.random_token(tuple(context_queue))
            result.append(obj)
            if n > 1:
                context_queue.pop(0)
                context_queue.append(obj)
        result = starting + result
        return ' '.join(result)


In [13]:
model = NgramModel(3)
model.update(words)

Testing the generative modeL, for some reason only works with certain starting words.


In [14]:
model.generate_text(100, ["quite","good"])

'quite good and the most comfortable beds great view from bar below prevented any sleep poor cleaning you have to walk luckily it wasn t cleaned when we were on their mobile during our stay hotel management can now do anything to make up for most of the hotel i am travelling for our pillow the following day to next room the light when we had to call me to get up and removed my still almost full plate of fruits as well is on the night manager extremely helpful best western agrees to give its guests freezing on arrival we were'

In [15]:
def top_sentence_SBERT(query,sentencecorpus,n):
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    flatsentencecorpus = [item for sublist in sentencecorpus for item in sublist] #flatten list as segmentation of document does not matter to BM25

    corpus_embeddings = embedder.encode(flatsentencecorpus, convert_to_tensor=True)

    # Find the closest n sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(n, len(flatsentencecorpus))
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    result=[]
    for score, idx in zip(top_results[0], top_results[1]):
        result.append([score,flatsentencecorpus[idx]])
    return result

In [16]:
sentence=[]
content=""
for text in reviews['Review']:
    sentencecontent = nltk.sent_tokenize(text)
    sentence.append(sentencecontent)
for text in reviews['Review'].iloc[:2000]:
    content+=text+"."
print(content)

 I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next day before 11 o clock in order to get the room i waned to Not the best way to begin yo

Testing Top_Sentence_SBERT. Works fairly well

In [17]:
top_sentence_SBERT("Advantages of the place?",sentence,3)

[[tensor(0.7523, device='cuda:0'), ' Convenience and location'],
 [tensor(0.7183, device='cuda:0'), ' Convenience of the location'],
 [tensor(0.7065, device='cuda:0'), ' Convenience of location']]

Testing Summarizer, doesn't work with large text corpus and just returns nothing

In [18]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(content, num_sentences=3))
print(full)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Rooms are nice but for elderly a bit difficult as most rooms are two story with narrow steps So ask for single level Inside the rooms are very very basic just tea coffee and boiler and no bar empty fridge . Had no dislikes found the hotel to be just what we needed . The room had a terrible view of a scruffy building and the windows were disgusting Lack of parking and staff said it would be OK to park where we did Ended up with a parking fine Ruined a birthday suprise I had planned for months for my partner Wouldn t stay again .


In [19]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(reviews, test_size=0.2)

In [21]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'Review'
LABEL_COLUMN = 'Labels'

In [22]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [23]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [24]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-c

In [25]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="./your_model_save_path", tokenizer=tokenizer, hub_model_id="8CSI/Travel_Sentimental_Analysis"
)

Cloning https://huggingface.co/8CSI/Travel_Sentimental_Analysis into local empty directory.


In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data,callbacks=push_to_hub_callback)

Epoch 1/2
2000/2000 [==============================] - 1756s 847ms/step - loss: 0.1391 - accuracy: 0.9523 - val_loss: 0.1285 - val_accuracy: 0.9551
Epoch 2/2
2000/2000 [==============================] - 1701s 850ms/step - loss: 0.0971 - accuracy: 0.9690 - val_loss: 0.1290 - val_accuracy: 0.9569


In [27]:
pred_sentences = ['Great hotel you know? It has all the things necessary to live',
                  'One star out of five, do not even know why i bothered coming here']

In [33]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
print(tf_predictions)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
print(label)
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

tf.Tensor(
[[0.00494257 0.99505746]
 [0.86919576 0.13080423]], shape=(2, 2), dtype=float32)
[1 0]
Great hotel you know? It has all the things necessary to live : 
 Positive
One star out of five, do not even know why i bothered coming here : 
 Negative


To load the models


In [32]:
 tokenizer = BertTokenizer.from_pretrained("8CSI/Travel_Sentimental_Analysis")
 model = TFBertForSequenceClassification.from_pretrained("8CSI/Travel_Sentimental_Analysis")

Some layers from the model checkpoint at 8CSI/Travel_Sentimental_Analysis were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at 8CSI/Travel_Sentimental_Analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
